In [3]:
import altair as alt
import numpy as np
import pandas as pd
import torch

from blase.optimizer import default_clean
from collections import defaultdict
from gollum.phoenix import PHOENIXSpectrum
from pathlib import Path
from re import split
from tqdm import tqdm

alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

# [2] **Line Count Heatmap**

In [4]:
line_stats = defaultdict(list)
for file in tqdm(Path('../../experiments/08_blase3D_HPC_test/emulator_states').glob('*0.0.pt'), total=657):
    state_dict = torch.load(file, 'cuda')
    tokens = split('[TGZ]', file.stem)
    line_stats['teff'].append(int(tokens[1]))
    line_stats['logg'].append(float(tokens[2]))
    line_stats['Z'].append(float(tokens[3]))
    line_stats['center'].append(state_dict['pre_line_centers'].cpu().numpy())
    line_stats['shift_center'].append(state_dict['lam_centers'].cpu().numpy())
    line_stats['amp'].append(state_dict['amplitudes'].cpu().numpy())
    line_stats['sigma'].append(state_dict['sigma_widths'].cpu().numpy())
    line_stats['gamma'].append(state_dict['gamma_widths'].cpu().numpy())
df = pd.DataFrame(line_stats).explode(['center', 'amp', 'sigma', 'gamma', 'shift_center'])
df_gp = pd.DataFrame(line_stats)[['teff', 'logg', 'Z']]
df['jitter'] = df.shift_center - df.center

100%|██████████| 657/657 [00:01<00:00, 366.58it/s]


In [5]:
alt.Chart(df.groupby(['teff', 'logg']).size().reset_index(name='n_lines'), width=1000, height=400).mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), y=alt.Y('logg:O', title='Surface Gravity'), color=alt.Color('n_lines:Q', title='Count', scale=alt.Scale(type='log')))\
    .configure_axis(labelFontSize=15, titleFontSize=24)\
    .configure_legend(labelFontSize=15, titleFontSize=15).save('images/figure2.png', scale_factor=4.0)

# [3, 4] **PHOENIX Subset Discrete Manifold**
Unknown line (Possibly C I) and He II line

In [6]:
target_line = df.value_counts('center').index[1]
(alt.Chart(df.query('center == @target_line'), width=1000, height=400, title=f'C I Line at {target_line} Å').mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity'), 
            color=alt.Color(f'amp:Q', title='Log-Amplitude')) &\
alt.Chart(df.query('center == @target_line'), width=1000, height=400, title=f'C I Line at {target_line} Å').mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity'), 
            color=alt.Color(f'sigma:Q', title='Gaussian Width')))\
    .configure_axis(labelFontSize=15, titleFontSize=24)\
    .configure_legend(labelFontSize=15, titleFontSize=15)\
    .configure_title(fontSize=25).resolve_scale(color='independent').save('images/figure3.png', scale_factor=4.0)

In [7]:
target_line = df.value_counts('center').index[3840]
white_bg = alt.Chart(df_gp, width=1000, height=400, title=f'He II Line at {target_line} Å').mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity', axis=alt.Axis(values=list(np.arange(2, 6, 0.5)))), 
            color=alt.value('white'))
amp_chart = alt.Chart(df.query('center == @target_line')).mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity'), 
            color=alt.Color(f'amp:Q', title='Log-Amplitude'))
sigma_chart = alt.Chart(df.query('center == @target_line')).mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity'), 
            color=alt.Color(f'sigma:Q', title='Gaussian Width'))
(white_bg + amp_chart & white_bg + sigma_chart) \
    .configure_axis(labelFontSize=15, titleFontSize=24)\
    .configure_legend(labelFontSize=15, titleFontSize=15)\
    .configure_title(fontSize=25).resolve_scale(color='independent').save('images/figure4.png', scale_factor=4.0)

# [5, 6] **PHOENIX Generator Continuous Manifold**

In [8]:
df = pd.read_parquet('../../experiments/10_end_to_end/interpolated_line.parquet.gz')
df.loc[:, 'sigma'][df.amp < -100] = np.nan
df.loc[:, 'amp'][df.amp < -100] = np.nan
df.logg = df.logg.round(1)
amp_chart = alt.Chart(df, width=1000, height=400, title=f'C I Line at 11617.66 Å').mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity', axis=alt.Axis(values=list(np.arange(2, 6, 0.5)))), 
            color=alt.Color('amp:Q', title='Log-Amplitude'))
sigma_chart = alt.Chart(df, width=1000, height=400, title=f'C I Line at 11617.66 Å').mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity', axis=alt.Axis(values=list(np.arange(2, 6, 0.5)))), 
            color=alt.Color('sigma:Q', title='Gaussian Width'))
(amp_chart & sigma_chart)\
    .configure_axis(labelFontSize=15, titleFontSize=24)\
    .configure_legend(labelFontSize=15, titleFontSize=15)\
    .configure_title(fontSize=25).resolve_scale(color='independent').save('images/figure5.png', scale_factor=4.0)

In [9]:
df = pd.read_parquet('../../experiments/10_end_to_end/HeII_line.parquet.gz')
white_bg = alt.Chart(df, width=1000, height=400, title=f'He II Line at 10123.66 Å').mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity', axis=alt.Axis(values=list(np.arange(2, 6, 0.5)))), 
            color=alt.value('white'))
df.loc[:, 'sigma'][df.amp < -100] = np.nan
df.loc[:, 'amp'][df.amp < -100] = np.nan
df.logg = df.logg.round(1)
amp_chart = alt.Chart(df).mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity', axis=alt.Axis(values=list(np.arange(2, 6, 0.5)))), 
            color=alt.Color('amp:Q', title='Log-Amplitude'))
sigma_chart = alt.Chart(df).mark_rect()\
    .encode(x=alt.X('teff:O', title='Effective Temperature [K]', axis=alt.Axis(values=list(np.arange(2400, 12001, 200)))), 
            y=alt.Y('logg:O', title='Surface Gravity', axis=alt.Axis(values=list(np.arange(2, 6, 0.5)))), 
            color=alt.Color('sigma:Q', title='Gaussian Width'))
(white_bg + amp_chart & white_bg + sigma_chart)\
    .configure_axis(labelFontSize=15, titleFontSize=24)\
    .configure_legend(labelFontSize=15, titleFontSize=15)\
    .configure_title(fontSize=25).resolve_scale(color='independent').save('images/figure6.png', scale_factor=4.0)

# [7] **Spectral Reconstruction Demo**

In [13]:
lo = 8561.9
hi = 8562.3
s1 = default_clean(PHOENIXSpectrum(teff=5700, logg=4.5))
s2 = default_clean(PHOENIXSpectrum(teff=5800, logg=4.5))
s3 = pd.read_parquet('../../experiments/10_end_to_end/reconstruction_demo.parquet.gz').query('wavelength <= @hi and wavelength >= @lo')
s3['Stellar Parameters'] = 'T: 5772 K, log(g): 4.4374, [Fe/H]: 0.0'
c1 = alt.Chart(pd.DataFrame({'wavelength': s1.wavelength.value, 'flux': s1.flux.value, 'Stellar Parameters': 'T: 5700 K, log(g): 4.5, [Fe/H]: 0.0'}).query('wavelength <= @hi and wavelength >= @lo'), width=1000, height=400, title='Fe I Line at 8562.09 Å').mark_line().encode(x=alt.X('wavelength', title='Wavelength [Å]'), y=alt.Y('flux', title='Normalized Flux').scale(zero=False), color='Stellar Parameters')
c2 = alt.Chart(pd.DataFrame({'wavelength': s2.wavelength.value, 'flux': s2.flux.value, 'Stellar Parameters': 'T: 5800 K, log(g): 4.5, [Fe/H]: 0.0'}).query('wavelength <= @hi and wavelength >= @lo')).mark_line().encode(x='wavelength', y='flux', color=alt.Color('Stellar Parameters', scale=alt.Scale(scheme='magma')))
c3 = alt.Chart(s3).mark_line().encode(x='wavelength', y='flux', color='Stellar Parameters')
(c1 + c2 + c3).configure_axis(labelFontSize=15, titleFontSize=24).configure_legend(labelFontSize=15, titleFontSize=15).configure_title(fontSize=25).configure_legend(labelFontSize=16, titleFontSize=16, padding=5, fillColor='white', orient='bottom-right', labelLimit=0).save('images/figure7.png', scale_factor=4.0)

# [8] **PHOENIX Generator Reconstruction Performance**

In [11]:
df = pd.DataFrame({'N': [2, 5, 10, 25, 50, 75, 100], 
                   'Pseudo-Parallel': [16.572938919067383, 18.667826108634472, 21.52281529456377, 28.28887704014778, 39.83675589412451, 53.722645066678524, 62.9894732311368],
                   'Serial': [29.681056290864944, 80.29932584613562, 156.88104890286922, 381.16286554932594, 747.9954723417759, 1159.3977541476488, 1518.2086955830455]})
df['R'] = df['Serial'] / df['Pseudo-Parallel']
df2 = df.copy()
df2['Pseudo-Parallel'] /= df2.N 
df2['Serial'] /= df2.N
df = df.melt(id_vars='N', var_name='Set', value_name='Time')
df2 = df2.melt(id_vars='N', var_name='Set', value_name='Time')
c1 = alt.Chart(df.query('Set != "R"'), width=400, height=400).mark_line().encode(x=alt.X('N', title='', axis=alt.Axis(labels=False)), y=alt.Y('Time', title='Time [s]', scale=alt.Scale(domain=[16, 1600], type='log')), color=alt.Color('Set', title=' Reconstruction Type'))
c2 = alt.Chart(df.query('Set == "R"'), width=400, height=100).mark_line().encode(x=alt.X('N', title='Number of Reconstructions'), y=alt.Y('Time', title='Speedup'), color=alt.value('purple'))
c3 = alt.Chart(df2.query('Set != "R"'), width=400, height=400).mark_line().encode(x=alt.X('N', title='', axis=alt.Axis(labels=False)), y=alt.Y('Time', title='Time per Spectrum [s]'), color=alt.Color('Set', title=' Reconstruction Type'))
alt.vconcat(c1, c3, c2, spacing=5).configure_axis(labelFontSize=15, titleFontSize=24).configure_legend(labelFontSize=16, titleFontSize=16, offset=-400, padding=5, fillColor='white').configure_title(fontSize=25).save('images/figure8.png', scale_factor=4.0)